In [20]:
import pandas as pd
import warnings
warnings.simplefilter(action='ignore')
import numpy as np

city_popu=pd.read_hdf('Processed_Data/citypopu.h5',key='raw2')
city_popu['城市']=city_popu['城市'].str.replace('市',"")
city_popu['常住人口']=pd.to_numeric(city_popu['常住人口'])

In [21]:
import os
dfo=pd.DataFrame()

for f in os.listdir('./DataSource/Huazhu/')[-5:]:
    print(f)
    date=f.split('_')[-1].split('.')[0]
    d=pd.read_csv('./DataSource/Huazhu\%s' % f)


    c1=d['brandLevel'].isin(['经济型','舒适型'])

    d1=d[c1][['cityId', 'cityName', 'brandLevel', 'hotelId', 'hotelName', 'roomName','checkInType', 'stock']]
    d1=d1.groupby(by=['cityName', 'brandLevel','hotelName','checkInType','roomName']).mean().reset_index()
    d1=d1.groupby(by=['cityName', 'brandLevel','checkInType','hotelName']).sum().reset_index()
    # pv=pd.pivot_table(d1,index='cityName',values='stock',columns='checkInType', aggfunc=np.sum).reset_index()
    pv=pd.pivot_table(d1,index=['cityName','hotelName'],values='stock',columns='checkInType', aggfunc=np.sum).reset_index()
    pv=pv[~pv['now'].isna()]
    pv=pv[~pv['future'].isna()]
    
    pv['bookedNights']=pv['future']-pv['now']
    pv['hotel_occ']=pv['bookedNights']/pv['future']
    
    c2=pv['now']>5
    c3=pv['hotel_occ']<0.9

    pv=pv[c2&c3]

    pv=pv.groupby(by='cityName').sum().reset_index()
    pv['city_occ']=pv['bookedNights']/pv['future']
    pv['date']=date

    dfo=pd.concat([dfo,pv],axis=0)

huazhu_stock_2022-12-22.csv
huazhu_stock_2022-12-23.csv
huazhu_stock_2022-12-24.csv
huazhu_stock_2022-12-25.csv
huazhu_stock_2022-12-26.csv


In [22]:
old=pd.read_hdf(r'Processed_Data/hotel.h5',key='1223')
new=pd.concat([old,dfo])
new=new.drop_duplicates()
new.to_hdf(r'hotel.h5',key='1223')

In [23]:
# df_covid=pd.read_hdf(r'BaiduFever.h5',key='1220')
# df_covid['BDfever_last7Avg']=df_covid.groupby('城市')['BDfever'].transform(lambda x: x.rolling(7, 7).mean())
# df_covid.tail(2)

df_covid=pd.read_hdf('Processed_Data/Baidu_covid_progress.h5',key='raw')
df_covid.tail(2)

,city,progress,peak,end,城市,常住人口,progress_2
295,聊城,回落至34%,2022/12/20,2023/03/07,聊城,201,34
296,宿迁,回落至41%,2022/12/21,2023/03/08,宿迁,310,41


In [27]:
# covid rank vs. hotel nights WoW
%load_ext autoreload
%autoreload 2
from colored_list import dafeng_city,jidui_city,riqi
# riqi='2022-12-26'
# hotel=pd.read_hdf(r'./hotel.h5',key='raw1')
hotel=new.copy()
hotel['date']=pd.to_datetime(hotel['date'])
hotel['cityName']=hotel['cityName'].str.replace('市','')

hotel1=hotel.merge(city_popu,left_on='cityName',right_on='城市',how='inner').merge(df_covid,left_on=['cityName'],right_on=['城市'],how='left')
hotel1['covid_rank']=hotel1.groupby(by=['date'])['progress_2'].rank(method="dense", ascending=False)

hotel1=hotel1.sort_values(by=['cityName','date'])
hotel1['bookedNights_last7d_mean']=hotel1.groupby('cityName')['bookedNights'].transform(lambda x: x.rolling(7, 7).mean())
hotel1['bookedNights_change_WoW']=(hotel1['bookedNights_last7d_mean']/hotel1.groupby('cityName')['bookedNights_last7d_mean'].transform(lambda x: x.shift(periods=7)))-1

hotel1['date']=hotel1['date'].astype('string') 

hotel2=hotel1[hotel1['date']==riqi].sort_values(by='bookedNights_last7d_mean',ascending=False)
hotel2=hotel2[hotel1['bookedNights_change_WoW']<2].head(100).reset_index()
# hote1
o=hotel2[['cityName',  'covid_rank','bookedNights_change_WoW']]


o['size']=np.nan
o['group']='C'

c1=o['cityName'].isin(dafeng_city)
o.loc[c1,'group']='A'

c2=o['cityName'].isin(jidui_city)
o.loc[c2,'group']='B'

o=o.sort_values(by='group')

o.to_clipboard(index=False,header=None)
print(riqi)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
2022-12-26


In [52]:
#calculate vs. 22-06

import datetime
hotel=new.copy()
hotel['date']=pd.to_datetime(hotel['date'])
hotel['cityName']=hotel['cityName'].str.replace('市','')


hotel['date']=pd.to_datetime(hotel['date'])
hotel['date']=hotel['date'].dt.date
hotel['bookedNights_7_mean']=hotel.groupby('cityName')['bookedNights'].transform(lambda x: x.rolling(7, 1).mean())


c1=hotel['date']< datetime.date(2022,7,1)
c2=hotel['bookedNights']>0
june_mean=hotel[c1&c2].groupby('cityName').mean().reset_index().sort_values(by='bookedNights')[['cityName','bookedNights']].reset_index(drop=True)
june_mean.columns=['cityName', 'bookedNights_june']
june_mean

hotel=hotel.merge(june_mean,on='cityName')
hotel['changes_since_june']=hotel['bookedNights_7_mean']/hotel['bookedNights_june']

c3=hotel['cityName'].isin(['北京','保定','石家庄','洛阳'])
show=hotel[c3]

show=pd.pivot_table(show,index='date',columns='cityName',values='changes_since_june',aggfunc='mean').reset_index()
show['sum']=np.nan

show=show[['date','sum','北京','保定','石家庄','洛阳']]
show.to_clipboard(index=False)

In [53]:
# covid rank vs. hotel nights vs 202206

hotel['date']=pd.to_datetime(hotel['date'])

hotel1=hotel.merge(city_popu,left_on='cityName',right_on='城市',how='inner').merge(df_covid,left_on=['cityName'],right_on=['城市'],how='left')
hotel1['covid_rank']=hotel1.groupby(by=['date'])['progress_2'].rank(method="dense", ascending=False)

hotel1=hotel1.sort_values(by=['cityName','date'])

hotel1['date']=hotel1['date'].astype('string') 

hotel2=hotel1[hotel1['date']==riqi].sort_values(by='bookedNights_7_mean',ascending=False)
hotel2=hotel2[hotel1['changes_since_june']<2].head(100).reset_index()
# hote1
o=hotel2[['cityName',  'covid_rank','changes_since_june']]


o['size']=np.nan
o['group']='C'

c1=o['cityName'].isin(dafeng_city)
o.loc[c1,'group']='A'

c2=o['cityName'].isin(jidui_city)
o.loc[c2,'group']='B'

o=o.sort_values(by='group')

o.to_clipboard(index=False,header=None)
print(riqi)

2022-12-26


In [54]:
d1=pd.read_clipboard()
d2=pd.pivot_table(dfo[dfo['date']>'2022-12-21'],index='cityName',columns='date',values='bookedNights',aggfunc='sum').reset_index()
d1.merge(d2,left_on='城市',right_on='cityName',how='left').to_clipboard()